In [1]:
import gym
import holdem
import agent

Using TensorFlow backend.


In [2]:
s = 3000
def add_users(env):
    model_list = list()

    # start with 4 players
    env.add_player(0, stack=s) # add a player to seat 0 with 1000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.dqnModel())

    env.add_player(1, stack=s) # add another player to seat 1 with 2000 "chips"
#     model_list.append(agent.CCModel())
    model_list.append(agent.dqnModel())

    env.add_player(2, stack=s) # add another player to seat 2 with 3000 "chips"
    # model_list.append(agent.allFoldModel())
#     model_list.append(agent.CCModel())
    model_list.append(agent.dqnModel())

    env.add_player(3, stack=s) # add another player to seat 3 with 1000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.dqnModel())

    env.add_player(4, stack=s) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.dqnModel())
#     model_list.append(agent.CCModel())
#     model_list.append(agent.MontecarloModel())

    env.add_player(5, stack=s) # add another player to seat 3 with 1000 "chips"
#     model_list.append(agent.dqnModel())
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.dqnModel())

    # env.add_player(6, stack=s) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allCallModel())

    # env.add_player(7, stack=s) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.MontecarloModel())

    # env.add_player(8, stack=s) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allFoldModel())

    # env.add_player(9, stack=s) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allinModel())

    return model_list

In [3]:
def lets_play(rounds=1):
    
    for i in range(rounds):
        print(i, " plays")
        _lets_play()
        
        
def _lets_play():
    env = gym.make('TexasHoldem-v2') # holdem.TexasHoldemEnv(2)
    model_list = add_users(env)
    
    while True:

        cur_state = env.reset()
        env.render(mode='human')
        cycle_terminal = False
        # (cur_state)
        if env.episode_end:
            break

        while not cycle_terminal:
            current_player = cur_state.community_state.current_player
            # play safe actions, check when no one else has raised, call when raised.
            # print(">>> Debug Information ")
            # print("state(t)")
            # for p in cur_state.player_states:
            #     print(p)
            # print(cur_state.community_state)

            actions = holdem.model_list_action(cur_state, n_seats=env.n_seats, model_list=model_list)
            cur_state, rews, cycle_terminal, info = env.step(actions)

            if cycle_terminal:
                for p in cur_state.player_states:
                    if p.emptyplayer: 
                        continue
                    model_action = model_list[p.seat].RoundEndAction(cur_state, p.seat)
#                 try:
#                     model_action = model_list[current_player].RoundEndAction(self.get_current_state(), my_seat)
#                 except:
#                     pass
                
            # print("action(t), (CALL=1, RAISE=2, FOLD=3 , CHECK=0, [action, amount])")
            # print(actions)

            # print("reward(t+1)")
            # print(rews)
            # print("<<< Debug Information ")

                
            env.render(mode="human")
        # print("final state")
        # print(cur_state)

        # total_stack = sum([p.stack for p in env._seats])
        # if total_stack != 10000:
        #     return

    print("Episode End!!!")

In [4]:
# for i in range(30):
#     print(i, " plays")
#     lets_play()

In [ ]:
lets_play(rounds=4)

0  plays
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               37900     
_________________________________________________________________
dense_38 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_39 (Dense)             (None, 4)                 204       
Total params: 43,154
Trainable params: 43,154
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 100)               37900     
_________________________________________________________________
dense_41 (Dense)             (None, 50)                5050      
_______________________________________________________________

Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[A♣] stack: 3000
1 [5♣],[7♠] stack: 2990
2 [8♥],[J♦] stack: 2980
3 [2♠],[J♠] stack: 3000
4 [6♦],[9♣] stack: 3000
5 [Q♣],[5♥] stack: 3000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 50 >>>
last action by player 3:	- call, current bet: 20
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[A♣] stack: 3000
1 [5♣],[7♠] stack: 2990
2 [8♥],[J♦] stack: 2980
3 [2♠],[J♠] stack: 2980
4 [6♦],[9♣] stack: 3000
5 [Q♣],[5♥] stack: 3000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 70 >>>
last action by player 4:	- call, current bet: 20
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[A♣] stack: 3000
1 [5♣],[7♠] stack: 2990
2 [8♥],[J♦] stack: 2980
3 [2♠],[J♠] stack: 2980
4 [6♦],[9♣] stack: 2980
5 [Q♣],[5♥] stack: 3000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ]

{'loss': [425.0906677246094]}
Cycle 1, total pot: 3814 >>>
last action by player 0:	- call, current bet: 238
community:
- [J♣],[9♠],[4♠],[9♦],[  ] 
players:
0 [K♦],[A♣] stack: 2146
1 [5♣],[7♠] stack: 2384
2 [8♥],[J♦] stack: 2384
3 [2♠],[J♠] stack: 2980
4 [6♦],[9♣] stack: 2146
5 [Q♣],[5♥] stack: 2146
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [2.64241623878479]}
Cycle 1, total pot: 4290 >>>
last action by player 1:	^ raise, current bet: 476
community:
- [J♣],[9♠],[4♠],[9♦],[  ] 
players:
0 [K♦],[A♣] stack: 2146
1 [5♣],[7♠] stack: 1908
2 [8♥],[J♦] stack: 2384
3 [2♠],[J♠] stack: 2980
4 [6♦],[9♣] stack: 2146
5 [Q♣],[5♥] stack: 2146
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [824.8130493164062]}
Cycle 1, total pot: 5004 >>>
last action by player 2:	^ raise, current bet: 714
community:
- [J♣],[9♠],[4♠],[9♦],[  ] 
players:
0 [K♦],[A♣] stack: 2146
1 [5♣],[7♠] stack: 1908
2 [8♥],[J♦] stac

{'loss': [463.09381103515625]}
Cycle 2, total pot: 120 >>>
last action by player 2:	_ check
community:
- [J♣],[T♣],[Q♠],[  ],[  ] 
players:
0 [8♠],[5♥] stack: 2126
1 [3♦],[9♠] stack: 1483
2 [A♦],[K♥] stack: 7059
3 [A♥],[T♦] stack: 2960
4 [8♣],[8♥] stack: 2126
5 [Q♣],[6♥] stack: 2126
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [189.4322967529297]}
Cycle 2, total pot: 238 >>>
last action by player 3:	^ raise, current bet: 118
community:
- [J♣],[T♣],[Q♠],[  ],[  ] 
players:
0 [8♠],[5♥] stack: 2126
1 [3♦],[9♠] stack: 1483
2 [A♦],[K♥] stack: 7059
3 [A♥],[T♦] stack: 2842
4 [8♣],[8♥] stack: 2126
5 [Q♣],[6♥] stack: 2126
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [322.2508239746094]}
Cycle 2, total pot: 356 >>>
last action by player 4:	- call, current bet: 118
community:
- [J♣],[T♣],[Q♠],[  ],[  ] 
players:
0 [8♠],[5♥] stack: 2126
1 [3♦],[9♠] stack: 1483
2 [A♦],[K♥] stack: 7059
3 [A♥],[T♦]

{'loss': [265.2952880859375]}
Cycle 2, total pot: 1988 >>>
last action by player 0:	_ check
community:
- [J♣],[T♣],[Q♠],[3♣],[4♥] 
players:
0 [8♠],[5♥] stack: 1648
1 [3♦],[9♠] stack: 1365
2 [A♦],[K♥] stack: 6581
3 [A♥],[T♦] stack: 2842
4 [8♣],[8♥] stack: 1928
5 [Q♣],[6♥] stack: 1648
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [575.9624633789062]}
Cycle 2, total pot: 1988 >>>
last action by player 2:	_ check
community:
- [J♣],[T♣],[Q♠],[3♣],[4♥] 
players:
0 [8♠],[5♥] stack: 1648
1 [3♦],[9♠] stack: 1365
2 [A♦],[K♥] stack: 6581
3 [A♥],[T♦] stack: 2842
4 [8♣],[8♥] stack: 1928
5 [Q♣],[6♥] stack: 1648
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [212.02725219726562]}
Cycle 2, total pot: 1988 >>>
last action by player 5:	_ check
community:
- [J♣],[T♣],[Q♠],[3♣],[4♥] 
players:
0 [8♠],[5♥] stack: 1648
1 [3♦],[9♠] stack: 1365
2 [A♦],[K♥] stack: 6581
3 [A♥],[T♦] stack: 2842
4 [8♣],[8♥] stack: 

...

Cycle 9, total pot: 724 >>>
last action by player 4:	x fold
community:
- [J♣],[T♣],[5♦],[  ],[  ] 
players:
0 [T♥],[Q♦] stack: 295
1 [K♣],[A♣] stack: 831
2 [3♠],[6♥] stack: 1501
3 [6♦],[2♥] stack: 8742
4 [2♦],[K♦] stack: 832
5 [9♠],[J♥] stack: 5075
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [48.84859848022461]}
Cycle 9, total pot: 724 >>>
last action by player 5:	_ check
community:
- [J♣],[T♣],[5♦],[Q♠],[  ] 
players:
0 [T♥],[Q♦] stack: 295
1 [K♣],[A♣] stack: 831
2 [3♠],[6♥] stack: 1501
3 [6♦],[2♥] stack: 8742
4 [2♦],[K♦] stack: 832
5 [9♠],[J♥] stack: 5075
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [4.370080947875977]}
Cycle 9, total pot: 724 >>>
last action by player 5:	_ check
community:
- [J♣],[T♣],[5♦],[Q♠],[  ] 
players:
0 [T♥],[Q♦] stack: 295
1 [K♣],[A♣] stack: 831
2 [3♠],[6♥] stack: 1501
3 [6♦],[2♥] stack: 8742
4 [2♦],[K♦] stack: 832
5 [9♠],[J♥] stack: 5075
6 [  ],[  ] sta

{'loss': [6.321920394897461]}
Cycle 10, total pot: 481 >>>
last action by player 1:	x fold
community:
- [3♠],[Q♣],[3♥],[  ],[  ] 
players:
0 [J♥],[2♥] stack: 295
1 [Q♠],[7♣] stack: 791
2 [5♥],[T♣] stack: 1401
3 [6♣],[4♥] stack: 8702
4 [8♦],[4♠] stack: 792
5 [A♣],[8♠] stack: 5538
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [10.476943016052246]}
Cycle 10, total pot: 481 >>>
last action by player 2:	x fold
community:
- [3♠],[Q♣],[3♥],[  ],[  ] 
players:
0 [J♥],[2♥] stack: 295
1 [Q♠],[7♣] stack: 791
2 [5♥],[T♣] stack: 1401
3 [6♣],[4♥] stack: 8702
4 [8♦],[4♠] stack: 792
5 [A♣],[8♠] stack: 5538
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [40.19254684448242]}
Cycle 10, total pot: 762 >>>
last action by player 3:	- call, current bet: 281
community:
- [3♠],[Q♣],[3♥],[  ],[  ] 
players:
0 [J♥],[2♥] stack: 295
1 [Q♠],[7♣] stack: 791
2 [5♥],[T♣] stack: 1401
3 [6♣],[4♥] stack: 8421
4 [8♦],[4♠] 

{'loss': [8.931391716003418]}
Cycle 11, total pot: 240 >>>
last action by player 1:	_ check
community:
- [A♦],[6♠],[5♦],[  ],[  ] 
players:
0 [8♥],[7♥] stack: 255
1 [J♦],[9♥] stack: 751
2 [K♦],[7♦] stack: 1361
3 [7♣],[T♦] stack: 9573
4 [T♠],[9♣] stack: 752
5 [2♣],[7♠] stack: 5068
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [4.8925700187683105]}
Cycle 11, total pot: 446 >>>
last action by player 5:	^ raise, current bet: 206
community:
- [A♦],[6♠],[5♦],[  ],[  ] 
players:
0 [8♥],[7♥] stack: 255
1 [J♦],[9♥] stack: 751
2 [K♦],[7♦] stack: 1361
3 [7♣],[T♦] stack: 9573
4 [T♠],[9♣] stack: 752
5 [2♣],[7♠] stack: 4862
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
{'loss': [9.476990699768066]}
Cycle 11, total pot: 446 >>>
last action by player 0:	x fold
community:
- [A♦],[6♠],[5♦],[  ],[  ] 
players:
0 [8♥],[7♥] stack: 255
1 [J♦],[9♥] stack: 751
2 [K♦],[7♦] stack: 1361
3 [7♣],[T♦] stack: 9573
4 [T♠],[9♣

Cycle 12, total pot: 100 >>>
last action by player 2:	- call, current bet: 40
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[T♥] stack: 235
1 [Q♦],[7♠] stack: 505
2 [7♣],[K♦] stack: 921
3 [A♥],[2♠] stack: 11005
4 [3♥],[J♣] stack: 752
5 [4♣],[5♦] stack: 4482
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 12, total pot: 140 >>>
last action by player 3:	- call, current bet: 40
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[T♥] stack: 235
1 [Q♦],[7♠] stack: 505
2 [7♣],[K♦] stack: 921
3 [A♥],[2♠] stack: 10965
4 [3♥],[J♣] stack: 752
5 [4♣],[5♦] stack: 4482
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 12, total pot: 140 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♥],[T♥] stack: 235
1 [Q♦],[7♠] stack: 505
2 [7♣],[K♦] stack: 921
3 [A♥],[2♠] stack: 10965
4 [3♥],[J♣] stack: 752
5 [4♣],[5♦] stack: 4482
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 

Cycle 13, total pot: 100 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [7♥],[A♠] stack: 235
1 [3♠],[4♦] stack: 485
2 [9♥],[K♦] stack: 789
3 [K♥],[8♠] stack: 11197
4 [9♦],[5♦] stack: 752
5 [Q♥],[7♠] stack: 4442
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 13, total pot: 140 >>>
last action by player 5:	- call, current bet: 40
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [7♥],[A♠] stack: 235
1 [3♠],[4♦] stack: 485
2 [9♥],[K♦] stack: 789
3 [K♥],[8♠] stack: 11197
4 [9♦],[5♦] stack: 752
5 [Q♥],[7♠] stack: 4402
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 13, total pot: 140 >>>
last action by player 0:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [7♥],[A♠] stack: 235
1 [3♠],[4♦] stack: 485
2 [9♥],[K♦] stack: 789
3 [K♥],[8♠] stack: 11197
4 [9♦],[5♦] stack: 752
5 [Q♥],[7♠] stack: 4402
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] sta